In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Sample data
user_likes = {
    'user_id': [1, 1, 2, 2, 3, 3, 3],
    'post_id': [101, 102, 101, 103, 102, 104, 105]
}
 
# Posts metadata (e.g., categories, tags, or text descriptions)
posts = {
    'post_id': [101, 102, 103, 104, 105],
    'tags': ['travel beach', 'food recipe', 'fitness health', 'tech gadgets', 'travel adventure']
}

# Convert data to DataFrames
user_likes_df = pd.DataFrame(user_likes)
posts_df = pd.DataFrame(posts)

# Merge the dataframes to associate users with post metadata
merged_df = user_likes_df.merge(posts_df, on='post_id')
print(merged_df)

# Create a post-content matrix
count_vectorizer = CountVectorizer()
post_content_matrix = count_vectorizer.fit_transform(posts_df['tags'])

# Calculate post similarity using cosine similarity
post_similarity = cosine_similarity(post_content_matrix, post_content_matrix)
print(post_similarity)

# Recommendation function
def recommend_posts(user_id, user_likes_df, posts_df, post_similarity, top_n=5):
    # Get posts liked by the user
    user_posts = user_likes_df[user_likes_df['user_id'] == user_id]['post_id'].values
    
    # Get indices of liked posts
    post_indices = [posts_df[posts_df['post_id'] == post_id].index[0] for post_id in user_posts]
    
    # Calculate the mean similarity of all posts liked by the user
    user_similarity = np.mean(post_similarity[post_indices], axis=0)
    
    # Rank posts by similarity score
    recommended_indices = np.argsort(-user_similarity)
    
    # Filter out posts already liked by the user
    recommended_posts = [
        posts_df.iloc[idx]['post_id']
        for idx in recommended_indices
        if posts_df.iloc[idx]['post_id'] not in user_posts
    ]
    
    return recommended_posts[:top_n]

# Example usage
user_id = 1
recommended_posts = recommend_posts(user_id, user_likes_df, posts_df, post_similarity, top_n=3)
print(f"Recommended posts for user {user_id}: {recommended_posts}")


   user_id  post_id              tags
0        1      101      travel beach
1        1      102       food recipe
2        2      101      travel beach
3        2      103    fitness health
4        3      102       food recipe
5        3      104      tech gadgets
6        3      105  travel adventure
[[1.  0.  0.  0.  0.5]
 [0.  1.  0.  0.  0. ]
 [0.  0.  1.  0.  0. ]
 [0.  0.  0.  1.  0. ]
 [0.5 0.  0.  0.  1. ]]
Recommended posts for user 1: [np.int64(105), np.int64(103), np.int64(104)]


In [1]:
import sqlite3
import pandas as pd
entryno="2024CS10020"

# Path to your SQLite database file
db_path = r"C:\Users\DELL\Documents\kars\instance\kars.db"


# Connect to the SQLite database
conn = sqlite3.connect(db_path)


# Define your SQL query
query = "SELECT * FROM registration"
query1= "SELECT * FROM events"
query2= "SELECT * FROM course_events"
query3= "SELECT * FROM students_events"

# Load data into a pandas DataFrame
registration = pd.read_sql_query(query, conn)
events=pd.read_sql_query(query1, conn)
course_events=pd.read_sql_query(query2, conn)
students_events=pd.read_sql_query(query3, conn)
# Close the connection
conn.close()
registration=registration[registration["entryno"]=="2024CS10020"]
students_events=students_events[students_events["entryno"]=="2024CS10020"]
# Display the DataFrame
# print(events)
# print(students_events)
print(registration)
# print(course_events)

       entryno          name                 email password  \
0  2024CS10020  rakshit modi  cs1240020@iitd.ac.in      rak   

                                                role department  \
0  ["student", "club", "department", "fest", "cou...       null   

                           photo                            interest  
0  WIN_20241109_02_47_00_Pro.jpg  ["reading", "traveling", "coding"]  


In [2]:
import json
user_entryno=registration["entryno"]

# Convert stored interests from JSON string to list
user_interests = json.loads(registration["interest"])

# Fetch all available events from the database
all_events = events.query.all()

if not all_events:
    return []  # Return empty if no events exist

# Prepare event data
event_names = [event.name for event in all_events]
event_tags = [event.tags for event in all_events]  # Tags are used for recommendation

# Convert user interests and event tags into text format for vectorization
all_texts = [' '.join(user_interests)] + event_tags

# Convert text data to TF-IDF vectors
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Compute cosine similarity between user interests (first row) and all event tags
similarity_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()

# Get the top N recommended event indices
top_indices = similarity_scores.argsort()[::-1][:top_n]

# Get event names based on top indices
recommended_events = [event_names[i] for i in top_indices]

return recommended_events


SyntaxError: 'return' outside function (3603840180.py, line 5)